# Checkpoint

## Persisting to disk

Spark uses lazy evaluation. As we build up many transformations Spark creates an execution plan for the DataFrame and the plan is executed when an action is called. This execution plan represents the DataFrame's lineage.

Sometimes the DataFrame's lineage can grow long and complex, which will slow down the processing and maybe even return an error. However, we can get around this by breaking the lineage.

There is more than one way of breaking the lineage, this is discussed in more detail in the **Persistence** article. In this article we cover a simple method of persisting to disk called checkpointing, which is essentially an out of the box shortcut to a write/read operation.

## Experiment

To demonstrate the benefit of checkpointing we'll time how long it takes to create a DataFrame using an iterative calculation. We will run the process without persisting, then again using a checkpoint. 

We'll create a new Spark session each time just in case there's an advantage when processing the DataFrame a second time in the same session. We will also use the Python module `time` to measure the time taken to create the DataFrame. 

We're going to create a new DataFrame with an `id` column and a column called `col_0` that will consist of random numbers. We'll then create a loop to add new columns where the values depend on a previous column. The contents of the columns isn't important here. What is important is that Spark is creating an execution plan that it getting longer with each iteration of the loop.

In general, we try to avoid using loops with Spark and this example shows why. A better solution to this problem using Spark would be to add new rows with each iteration as opposed to columns.

We will set a `seed_num` when creating the random numbers to make the results repeatable. The DataFrame will have `num_rows` amount of rows, which we will set to a thousand and the loop will iterate 11 times to create `col_1` to `col_11`.

In [1]:
import os
from pyspark.sql import SparkSession, functions as F
from time import time
import yaml

spark = (SparkSession.builder.master("local[2]")
         .appName("checkpoint")
         .getOrCreate())

new_cols = 12
seed_num = 42
num_rows = 10**3

### Without persisting

In [3]:
start_time = time()

df = spark.range(num_rows)
df = df.withColumn("col_0", F.ceil(F.rand(seed_num) * new_cols))

for i in range(1, new_cols):
    df = (df.withColumn("col_"+str(i), 
                        F.when(F.col("col_"+str(i-1)) > i, 
                               F.col("col_"+str(i-1)))
                        .otherwise(0)))

df.show(10)

time_taken = time() - start_time
print(f"Time taken to create the DataFrame:  {time_taken}")

+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+
| id|col_0|col_1|col_2|col_3|col_4|col_5|col_6|col_7|col_8|col_9|col_10|col_11|
+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+
|  0|    8|    8|    8|    8|    8|    8|    8|    8|    0|    0|     0|     0|
|  1|   11|   11|   11|   11|   11|   11|   11|   11|   11|   11|    11|     0|
|  2|   11|   11|   11|   11|   11|   11|   11|   11|   11|   11|    11|     0|
|  3|   11|   11|   11|   11|   11|   11|   11|   11|   11|   11|    11|     0|
|  4|    6|    6|    6|    6|    6|    6|    0|    0|    0|    0|     0|     0|
|  5|    7|    7|    7|    7|    7|    7|    7|    0|    0|    0|     0|     0|
|  6|    1|    0|    0|    0|    0|    0|    0|    0|    0|    0|     0|     0|
|  7|    2|    2|    0|    0|    0|    0|    0|    0|    0|    0|     0|     0|
|  8|    4|    4|    4|    4|    0|    0|    0|    0|    0|    0|     0|     0|
|  9|    9|    9|    9|    9|    9|    9

The result above shows how long Spark took to create the plan and execute it to show the top 10 rows. 

### With checkpoints

Next we will stop the Spark session and start a new one to repeat the operation using checkpoints. 

To perform a checkpoint we need to set up a checkpoint directory on the file system, which is where the checkpointed DataFrames will be stored. It's important to practice good housekeeping with this directory because new files are created with every checkpoint, but they are **not automatically deleted**.

In [4]:
spark.stop()

spark = (SparkSession.builder.master("local[2]")
         .appName("checkpoint")
         .getOrCreate())

with open("../../../config.yaml") as f:
    config = yaml.safe_load(f)
    
checkpoint_path = config["checkpoint_path"]
spark.sparkContext.setCheckpointDir(checkpoint_path)

We will checkpoint the DataFrame every 3 iterations of the loop so that the lineage doesn't grow as long. Again, we will time how long it takes for Spark to complete the operation.

In [5]:
start_time = time()

df = spark.range(num_rows)
df = df.withColumn("col_0", F.ceil(F.rand(seed_num) * new_cols))

for i in range(1, new_cols):
    df = (df.withColumn("col_"+str(i), 
                       F.when(F.col("col_"+str(i-1)) > i, 
                              F.col("col_"+str(i-1)))
                       .otherwise(0)))
    if i % 3 == 0: # this means if i is divisable by three then...
        df = df.checkpoint() # here is the checkpoint
        
df.show(10)

time_taken = time() - start_time
print(f"Time taken to create the DataFrame:  {time_taken}")

+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+
| id|col_0|col_1|col_2|col_3|col_4|col_5|col_6|col_7|col_8|col_9|col_10|col_11|
+---+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+
|  0|    8|    8|    8|    8|    8|    8|    8|    8|    0|    0|     0|     0|
|  1|   11|   11|   11|   11|   11|   11|   11|   11|   11|   11|    11|     0|
|  2|   11|   11|   11|   11|   11|   11|   11|   11|   11|   11|    11|     0|
|  3|   11|   11|   11|   11|   11|   11|   11|   11|   11|   11|    11|     0|
|  4|    6|    6|    6|    6|    6|    6|    0|    0|    0|    0|     0|     0|
|  5|    7|    7|    7|    7|    7|    7|    7|    0|    0|    0|     0|     0|
|  6|    1|    0|    0|    0|    0|    0|    0|    0|    0|    0|     0|     0|
|  7|    2|    2|    0|    0|    0|    0|    0|    0|    0|    0|     0|     0|
|  8|    4|    4|    4|    4|    0|    0|    0|    0|    0|    0|     0|     0|
|  9|    9|    9|    9|    9|    9|    9

The exact times will vary with each run of this notebook, but hopefully you will see that using the `.checkpoint()` was more efficient.

As mentioned earlier, the checkpoint files are not deleted on HDFS automatically. The files are not intended to be used after you stop the Spark session, so make sure you delete these files after a session.

Often the easiest way to delete files is through some GUI, but the cell below is handy to have at the end of your scripts when using checkpoints to make sure you don't forget to empty the checkpoint folder.

In [6]:
import subprocess
cmd = f'hdfs dfs -rm -r -skipTrash {checkpoint_path}' 
p = subprocess.run(cmd, shell=True)

### How often should I checkpoint?

How did we come up with the number 3 for number of iterations to checkpoint? Trial and error. Unfortunately, you may not have the luxury of trying to find the optimum number, but have a go at checkpointing and see if you can get any improvements in performance.

More frequent checkpointing means more writing and reading data, which does take some time, but the aim is to save some time by simplifying the execution plan.

As mentioned above, the use of loops shown here is not considered good practice with Spark, but it was a convenient example of using checkpoints. Of course, checkpointing can also be used outside loops, see the **Persistence** article for more information on the different forms of persisting data in Spark and their applications.

### Documentation

[`.checkpoint()`](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=checkpoint#pyspark.sql.DataFrame.checkpoint)